In [1]:
from pathlib import Path

def find_requirements_files(start_dir="."):
    """
    Parcourt récursivement les dossiers pour trouver tous les 
    fichiers nommés 'requirements.txt'.
    """
    base_path = Path(start_dir)
    # rglob est un "recursive glob" : il cherche partout sous base_path
    requirements_paths = list(base_path.rglob("requirements.txt"))
    
    if not requirements_paths:
        print("⚠️ Aucun fichier requirements.txt trouvé.")
    else:
        print(f"🔍 {len(requirements_paths)} fichier(s) trouvé(s) :")
        for path in requirements_paths:
            print(f"  - {path}")
            
    return requirements_paths
def get_file_content(file_path):
    """
    Lit le contenu d'un fichier et retourne une liste de lignes.
    """
    try:
        with open(file_path, 'r') as file:
            content = file.readlines()
        return content
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier {file_path}: {e}")
        return []


In [2]:
path = find_requirements_files()
content = get_file_content(path[0])

🔍 1 fichier(s) trouvé(s) :
  - dossier_projet/requirements.txt


### Gestion des requêtes en fonction de la version du package 
+ Pour les packages type : 'nom == xx.xx.xx' , on lance la requête simplement
+ Pour les packages type : 'nom >= xx.xx.xx' , on lance la requête  sans version spécifique pour avoir les vulnérabilités générales du packet. Puisque pip aura à gérer la version du packet dynamiquement en fonction des dépendances des autres packets, il vaut mieux faire une analyse générale des packets en excluant les versions inférieurs à la version de base.
+ Pour les packages type : 'nom ~= xx.xx.xx' , on lance la requête  sans version spécifique pour avoir les vulnérabilités générales du packet. Puisque pip aura à gérer la version du packet dynamiquement en fonction des dépendances des autres packets, il vaut mieux faire une analyse générale des packets en excluant les versions qui ne sont pas compatibles avec la condition
+ Pour les packages type : 'nom' , on lance la requête  sans version spécifique pour avoir les vulnérabilités générales du packet. Puisque pip aura à gérer la version du packet dynamiquement en fonction des dépendances des autres packets, il vaut mieux faire une analyse générale des packets

In [3]:
import requests
import pandas as pd

def check_vulnerability(package_name, version, constraint=None):
    """Interroge l'API OSV pour un package et une version donnés."""
    url = "https://api.osv.dev/v1/query"
    payload = {
        "version": version,
        "package": {"name": package_name, "ecosystem": "PyPI"}
    }
    
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        data = response.json()
        # Si le dictionnaire 'vulns' existe, le package est vulnérable
        res = fetch_vulnerabilities_details(data.get("vulns", []))
        if res.empty:
            return res
        res["package"] = package_name
        res["version"] = version
        return filter_based_on_constraints(res, version, constraint)
        
    return pd.DataFrame()  # Retourne un DataFrame vide si pas de vulnérabilités

def fetch_vulnerabilities_details(vulnerabilities):
    """Récupère les détails de chaque vulnérabilité."""
    details = []
    for vuln in vulnerabilities:
        details.append({
            "id": vuln["id"],
            "summary": vuln["summary"] if "summary" in vuln else "No summary available",
            "details": vuln["details"] if "details" in vuln else "No details available",
            "severity": vuln["database_specific"]["severity"] if "database_specific" in vuln else "UNKNOWN",
            "aliases": vuln["aliases"] if "aliases" in vuln else [],
            "affected_versions": vuln["affected"][0]["versions"],
            "events": vuln["affected"][0]["ranges"][0]["events"] 
        })
    return pd.DataFrame(details)

def filter_based_on_constraints(vulnerabilities_df, version, constraint):
    """Filtre les vulnérabilités en fonction d'une contrainte de version."""
    if constraint == "minimum":
        # Filtre les vulnérabilités affectant des versions supérieures ou égales à la version donnée
        return vulnerabilities_df[vulnerabilities_df["affected_versions"].apply(lambda versions: any(v >= version for v in versions))]
    elif constraint == "compatible":
        # Filtre les vulnérabilités affectant des versions compatibles avec la version donnée
        return vulnerabilities_df[vulnerabilities_df["affected_versions"].apply(lambda versions: any(v.startswith(".".join(version.split(".")[:-1])) for v in versions))]
    elif constraint == "fixed":
        # Filtre les vulnérabilités affectant exactement la version donnée
        return vulnerabilities_df[vulnerabilities_df["affected_versions"].apply(lambda versions: version in versions)]
    else:
        # Si aucune contrainte n'est spécifiée, retourne toutes les vulnérabilités
        return vulnerabilities_df

def parse_requirements(requirements_content):
    """Parse le contenu d'un fichier requirements.txt et retourne une liste de tuples (package, version)."""
    dependencies = []
    for line in requirements_content.splitlines():
        line = line.strip()
        if line and not line.startswith("#") and not line.startswith("-r") and not line.startswith("git+"):
            if "==" in line:
                package, version = line.split("==")
                dependencies.append((package.strip(), version.strip(), "fixed"))
            elif ">=" in line:
                package, version = line.split(">=")
                dependencies.append((package.strip(), version.strip(), "minimum"))
            elif "~=" in line:
                package, version = line.split("~=")
                dependencies.append((package.strip(), version.strip(), "compatible"))
            else:
                dependencies.append((line, "", "latest"))
    return dependencies

def get_packages_vulnerabilities(package_list):
    """Vérifie les vulnérabilités pour une liste de packages et versions."""
    results = None
    for package, version, constraint in package_list:
        res = check_vulnerability(package, version, constraint)
        if results is None:
            results = res
        else:
            results = pd.concat([results, res], ignore_index=True)
            
    return results

def vulnerability_summary(vulnerabilities_df):
    """Affiche un résumé des vulnérabilités trouvées."""
    if vulnerabilities_df.empty:
        print("✅ Aucun package vulnérable trouvé.")
    else:
        print(f"⚠️ {len(vulnerabilities_df)} vulnérabilité(s) trouvée(s) :")
        # Affichage du nombre de vulnérabilités par sévérité
        severity_counts = vulnerabilities_df["severity"].value_counts()
        for severity, count in severity_counts.items():
            print(f"  - {severity}: {count} vulnérabilité(s)")
        # Affichage des packages liés aux sévérités High et Critical
        for severity in ["HIGH", "CRITICAL"]:
            if severity in severity_counts:
                print(f"\n📌 Packages avec des vulnérabilités {severity} :")
                for package in vulnerabilities_df[vulnerabilities_df["severity"] == severity]["package"].unique():
                    print(f"  - {package}")

In [4]:
requirements_file_content = "".join(content)
print(requirements_file_content)

# --- Dépendances avec versions figées (Le plus sûr pour la prod) ---
scikit-learn
requests==2.28.1
pandas==1.5.3
gunicorn==20.1.0

# --- Dépendances avec contraintes souples (Risque de "Drift") ---
# Installe la version 3.0.0 ou toute version supérieure
flask>=3.0.0

# Installe une version compatible (souvent utilisé pour éviter les ruptures)
# Ici : accepte 2.1.0, 2.1.1... mais PAS la 2.2.0
cryptography~=2.1.0


In [5]:
parsed_dependencies = parse_requirements(requirements_file_content)
parsed_dependencies

[('scikit-learn', '', 'latest'),
 ('requests', '2.28.1', 'fixed'),
 ('pandas', '1.5.3', 'fixed'),
 ('gunicorn', '20.1.0', 'fixed'),
 ('flask', '3.0.0', 'minimum'),
 ('cryptography', '2.1.0', 'compatible')]

In [6]:
vulnerabilites = get_packages_vulnerabilities(parsed_dependencies)

In [7]:
vulnerability_summary(vulnerabilites)

⚠️ 23 vulnérabilité(s) trouvée(s) :
  - HIGH: 8 vulnérabilité(s)
  - MODERATE: 6 vulnérabilité(s)
  - UNKNOWN: 6 vulnérabilité(s)
  - LOW: 2 vulnérabilité(s)
  - CRITICAL: 1 vulnérabilité(s)

📌 Packages avec des vulnérabilités HIGH :
  - scikit-learn
  - gunicorn
  - cryptography

📌 Packages avec des vulnérabilités CRITICAL :
  - scikit-learn
